<a href="https://colab.research.google.com/github/mar7i4ka/AI-Agent-Models/blob/main/Problem_Solving_by_State_Space_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Problem:
    def __init__(self, initial=None, goal=None):
        self.initial = initial
        self.goal    = goal

    def actions(self, state):
        raise NotImplementedError

    def result(self, state, action):
        raise NotImplementedError

    def goal_test(self, state):
        return state == self.goal

    def path_cost(self, cost_so_far, s1, action, s2):
        return cost_so_far + 1          # г-функция (натрупана цена)
    def h(self, node):
        return 0.0                      # оценка колко още остава

class Node:

    __slots__ = ("state", "parent", "action", "path_cost", "depth", "f") #може да има само тези атрибути

    def __init__(self, state, parent=None, action=None,
                 path_cost=0.0, depth=0):
        self.state     = state
        self.parent    = parent
        self.action    = action
        self.path_cost = path_cost
        self.depth     = depth #брой стъпки
        self.f         = 0.0            # ще бъде калкулирана от алгоритъма

    def __lt__(self, other):            #позволява възлите да са сравними (използва heapq)
        return self.f < other.f         # enough for min() to work

    def expand(self, problem):          #съзава деца на nodes
        return [self.child_node(problem, a) for a in problem.actions(self.state)]

    def child_node(self, problem, action):
        s2 = problem.result(self.state, action)
        g2 = problem.path_cost(self.path_cost, self.state, action, s2)
        return Node(s2, self, action, g2, self.depth + 1)

    def path(self):
        node, out = self, []
        while node:
            out.append(node)
            node = node.parent
        return list(reversed(out))

# Best first_graph_search stops the search and returns the current node as soon as goal_test reports True.
def best_first_graph_search(problem, f):
    start = Node(problem.initial) #създава възел
    start.f = f(start) #изчислява оценъчната функция
    frontier = [start]          # стздава отворен списък с възлите които ще се разглеждат
    explored = {}               # запазва в state най добрата path cost(g) до сега за да не се разглеждат по-лоши пътища

    while frontier:

        node = min(frontier, key=lambda n: n.f) #намира възела с най-малка f
        frontier.remove(node) #премахване от възела за да се разгледа

        if problem.goal_test(node.state):
            return node

        explored[node.state] = node.path_cost #маркиране на възела

        for child in node.expand(problem): #генерира всички възможни наследници и изчислява за всеки един f
            child.f = f(child)
            if (child.state not in explored or
                    child.path_cost < explored[child.state]):
                frontier.append(child)
                explored[child.state] = child.path_cost

    raise ValueError("Goal not reachable")

def astar_search(problem):
    """A* = best-first with f = g + h."""
    return best_first_graph_search(problem, f=lambda n: n.path_cost + problem.h(n)) #problem.h(n) манхатънско разстоянив

class MazeProblem(Problem):

    def __init__(self, maze, start, goal):
        super().__init__(start, goal)
        self.maze  = maze
        self.rows  = len(maze)
        self.cols  = len(maze[0])

    def actions(self, state):
        row, col = state
        for directionR, directionC in [(-1,0), (1,0), (0,-1), (0,1)]:
            newr, newc = row + directionR, col + directionC
            if (0 <= newr < self.rows and 0 <= newc < self.cols #вътре в решетката ли е агента
                    and self.maze[newr][newc] == 0): #свободна ли е клетката
                yield (newr, newc) #всички възможни клетки за движение (не ползвам return за да не се прави списък)

    def result(self, state, action):
        return action

    def path_cost(self, g_so_far, s1, action, s2):
        return g_so_far + 1                  # +1

    # Task 3 – heuristic (Манхатънско разстояние между текущата клетка (row, col) и целта (goalr, goalc):)
    def h(self, node):
        (row, col)   = node.state
        (goalr, goalc) = self.goal
        return abs(row - goalr) + abs(col - goalc)

# 0 = free, 1 = wall
MAZE = [

    [ 0, 1, 0, 0, 0, 0 ],
    [ 0, 0, 0, 0, 0, 0 ],
    [ 0, 1, 0, 1, 0, 0 ],
    [ 0, 1, 0, 0, 1, 0 ],
    [ 0, 0, 0, 0, 1, 0 ],
]

START = (0, 0)
GOAL  = (4, 5) #цел

def show_maze(maze, path=()):
    sym  = {0: "·", 1: "█"}
    path = set(path)
    for r, row in enumerate(maze):
        line = ""
        for c, cell in enumerate(row):
            p = (r, c)
            if   p == START: line += "S "
            elif p == GOAL:  line += "G "
            elif p in path:  line += "* "
            else:            line += sym[cell] + " "
        print(line)
    print()


def main():
    # Tasks 1-3
    problem = MazeProblem(MAZE, START, GOAL)

    # Task 4 – run A*
    goal_node = astar_search(problem)

    # Task 5 – print optimal path and cost
    path = [n.state for n in goal_node.path()]
    print("Optimal path:", path)
    print("Total cost  :", goal_node.path_cost)
    print()

    # Task 7 – visual confirmation
    show_maze(MAZE, path)

if __name__ == "__main__":
    main()



Optimal path: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (2, 4), (2, 5), (3, 5), (4, 5)]
Total cost  : 9.0

S █ · · · · 
* * * * * · 
· █ · █ * * 
· █ · · █ * 
· · · · █ G 

